# Quick view musicians movements
Concert 1, slow ACC



In [ ]:
import sys
import os
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import heartpy as hp

from scipy.signal import butter,filtfilt
from scipy import interpolate
from scipy.interpolate import interp1d

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport qex

In [ ]:
# location of the files when in one of the local project files
projectName = 'Lydo_C1_C2' # this needs to match the project folder name
path = "../Data/Equivital/"+projectName+"/"
sep = '/'

In [ ]:
df_datafiles = qex.qiosk_recordings(path,projectName,sep)
df_datafiles.set_index('DevName',inplace=True)

In [ ]:
player_order = ['VN102','VN107','VN106','VN108','VN101','VN105','VN103','VN104','VN205','VN203','VN202','VN206','VN209','VN210','VN207','VN208','VN201','VN204','VA305','VA307','VA308','VA301','VA306','VA302','VA303','VA304','LS406','LS407','LS408','LS405','LS401','LS403','LS402','LS410','LS404','WW505','WW502','WW503','WW506','WW501','WW504','WW507','BR604','BR603','BR602','BR606','BR607','BR605','BR601','PC704','PC705','PC703','PC701','PC702','PC707']
#player_order = ['VN102','VN107','VN106','VN108','VN101','VN105','VN103','VN104','VN205','VN203','VN202','VN206','VN209','VN210','VN207','VN208','VN201','VN204','VA305','VA307','VA308','VA301','VA306','VA302','VA303','VA304','LS406','LS407','LS408','LS405','LS401','LS403','LS402','LS404','WW505','WW502','WW503','WW506','WW501','WW504','WW507','BR604','BR603','BR602','BR606','BR607','BR605','BR601','PC704','PC705','PC703','PC701','PC702','PC707']
# 8,10,8,
# 'Violin 1'
# 'Violin 2'
len(player_order)

In [ ]:
df_datafiles = df_datafiles.loc[player_order]
df_datafiles

In [ ]:
# signal File type and column name (multi sample)
signal_file_type = 'DATA'
sigName = 'HR(BPM)'
#sigName = 'TEMPERATURE'
sample_period = 15 # seconds 
interval_name = 'Lydo_1'
s_time = pd.to_datetime('2023-02-16 8:30:00+0000')
e_time = pd.to_datetime('2023-02-16 9:40:00+0000')
dur = (e_time - s_time).total_seconds()
time_s =  np.round(np.arange(0,dur,sample_period),3)

time_dt = s_time + pd.to_timedelta(time_s,unit='s')
df_sig = pd.DataFrame(index = time_dt.format(formatter=lambda x: x.strftime('%H:%M:%S')))

In [ ]:
3900/60

In [ ]:
# signal File type and column name (multi sample)
signal_file_type = 'DATA'
sigName = 'HR(BPM)'
#sigName = 'TEMPERATURE'
sample_period = 15 # seconds 
interval_name = 'Lydo_2'
s_time = pd.to_datetime('2023-02-16 10:15:00+0000')
e_time = pd.to_datetime('2023-02-16 11:15:00+0000')
dur = (e_time - s_time).total_seconds()
time_s =  np.round(np.arange(0,dur,sample_period),3)

time_dt = s_time + pd.to_timedelta(time_s,unit='s')
df_sig = pd.DataFrame(index = time_dt.format(formatter=lambda x: x.strftime('%H:%M:%S')))

In [ ]:
s_files = df_datafiles.copy()
# recordings started before the recording interval
s_files=s_files.loc[s_files['RecStart']< s_time,:] 
# recordings ended after the recording interval
s_files=s_files.loc[s_files['RecEnd']> e_time,:] 
print(len(s_files))

In [ ]:
# for most signals
for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    PartName = i

    sigFileLoc = []
    matched = qex.matched_files(dataFile,path,sep)
    for f in matched:
        if signal_file_type in f:
            sigFileLoc = f

    if sigFileLoc:
        V = pd.read_csv(sigFileLoc,skipinitialspace=True)
        V['DateTime'] = pd.to_datetime(V['DateTime'])
        sig_t = (V['DateTime'] - s_time).dt.total_seconds()
        cols = V.columns
        if sigName in cols:
            sig_v = V[sigName]
            f = interpolate.interp1d(sig_t, sig_v,fill_value='extrapolate')
            new_sig = f(time_s)
         #   if pd.Series(new_sig).diff().abs().sum()/len(new_sig)>1: # since th
            df_sig.loc[:,PartName] = new_sig

In [ ]:
Concert_times = pd.read_csv('./Timing/claps_lydo1_norge.csv',sep=",") #SV alignment for beeps
Concert_times['TIME'] = 60*Concert_times['Minutes']+Concert_times['Seconds']
Concert_times

In [ ]:
new_yticks = [0,8,18,26,31,35,43,49,54]
ytickLabels=['Violin 1','Violin 2','Viola','Cello','Bass','Woodwinds','Brass','Perc.','Conductor']


In [ ]:
fig, (ax1) = plt.subplots(1, 1,figsize = [15,6])
df_A = df_sig.diff()
df_A.set_index(time_s,inplace=True)
chart = sns.heatmap(data=df_A.transpose(), ax=ax1, vmin=-4, vmax=4, cbar=True,cmap="bwr",alpha=0.8)
#ax1.set_yticks([])
#ax1.set_yticklabels([0,10])
subtimes= Concert_times
new_xticks = len(df_A)*(subtimes.TIME-df_A.index[0])/(df_A.index[-1]-df_A.index[0])
ax1.set_xticks(new_xticks)
ax1.set_xticklabels(subtimes.Event)
ax1.set_yticks(new_yticks)
ax1.set_yticklabels(ytickLabels)
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Sections')
plt.grid(color='k')
plt.title('Change in Heart rates (BPM) across musicians for first Lydo concert')
plt.savefig('./plots/'+projectName+'_'+ interval_name+ '_set_'+sigName+'.png',bbox_inches = 'tight',dpi = 300)
plt.show()

In [ ]:
fig, (ax1) = plt.subplots(1, 1,figsize = [20,6])
chart = sns.heatmap(data=df_sig.diff().transpose(), ax=ax1, vmin=-0.15, vmax=0.15, cbar=True,cmap="spring",alpha=0.8)
ax1.set_xticks(new_xticks)
ax1.set_xticklabels(subtimes.Event)
ax1.set_yticks(new_yticks)
ax1.set_yticklabels(ytickLabels)
ax1.set_ylabel('Time (s)')
plt.grid(color='k')
plt.title('Change in ' + sigName + ' for interval ' + interval_name)
plt.savefig('./plots/'+projectName+'_'+ interval_name+ '_set_'+sigName+'.png',dpi = 300)
plt.show()

Need to synchronise recording times to higher precision for other signals

In [ ]:
# signal File type and column name (multi sample)
signal_file_type = 'RESPACC'
sigName = 'Breathing'
sample_period = 0.1 # seconds 
interval_name = 'Lydo_1'
s_time = pd.to_datetime('2023-02-16 8:30:00+0000')
e_time = pd.to_datetime('2023-02-16 9:30:00+0000')
dur = (e_time - s_time).total_seconds()
time_s =  np.round(np.arange(0,dur,sample_period),3)

time_dt = s_time + pd.to_timedelta(time_s,unit='s')
df_sig = pd.DataFrame(index = time_dt.format(formatter=lambda x: x.strftime('%H:%M:%S')))

In [ ]:
# for most signals
for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    PartName = i

    sigFileLoc = []
    matched = qex.matched_files(dataFile,path,sep)
    for f in matched:
        if signal_file_type in f:
            sigFileLoc = f

    if sigFileLoc:
        V = pd.read_csv(sigFileLoc,skipinitialspace=True)
        V['DateTime'] = pd.to_datetime(V['DateTime'])
        sig_t = (V['DateTime'] - s_time).dt.total_seconds()
        cols = V.columns
        if sigName in cols:
            sig_v = V[sigName]
            f = interpolate.interp1d(sig_t, sig_v,fill_value='extrapolate')
            new_sig = f(time_s)
         #   if pd.Series(new_sig).diff().abs().sum()/len(new_sig)>1: # since th
            df_sig.loc[:,PartName] = new_sig

In [ ]:
fig, (ax1) = plt.subplots(1, 1,figsize = [20,6])
chart = sns.heatmap(data=df_sig.diff().transpose(), ax=ax1,vmin=-20, vmax=20,  cbar=True,cmap="bwr",alpha=0.8)

# chart = sns.heatmap(data=df_sig.transpose(), ax=ax1,vmin=40, vmax=120, cbar=True, cmap="YlGnBu",alpha=0.8)
#ax1.set_yticks([])
plt.title(projectName + ' Change in ' + sigName + ' for interval ' + interval_name)
plt.savefig('./plots/'+projectName+'_'+ interval_name+ '_set_'+sigName+'.png',dpi = 300)
plt.show()

# Synchronisation cue

In [ ]:
cols

In [ ]:
# signal File type and column name (multi sample)
signal_file_type = 'FASTACC'
sample_period = 0.05 # seconds 
interval_name = 'Lydo_1_Full'
sigName = 'BodySway'
s_time = pd.to_datetime('2023-02-16 8:30:00+0000')
e_time = pd.to_datetime('2023-02-16 9:30:00+0000')
dur = (e_time - s_time).total_seconds()
time_s =  np.round(np.arange(0,dur,sample_period),3)

time_dt = s_time + pd.to_timedelta(time_s,unit='s')
df_sig = pd.DataFrame(index = time_dt.format(formatter=lambda x: x.strftime('%H:%M:%S')))

In [ ]:
interval_name = 'Lydo_1_Full'


In [ ]:
s_files = df_datafiles.copy()
# recordings started before the recording interval
s_files=s_files.loc[s_files['RecStart']< s_time,:] 
# recordings ended after the recording interval
s_files=s_files.loc[s_files['RecEnd']> e_time,:] 
print(len(s_files))

In [ ]:
sigName = 'Acc'

In [ ]:
# for acceleration
for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    PartName = i

    sigFileLoc = []
    matched = qex.matched_files(dataFile,path,sep)
    for f in matched:
        if signal_file_type in f:
            sigFileLoc = f

    if sigFileLoc:
        V = pd.read_csv(sigFileLoc,skipinitialspace=True)
        V['DateTime'] = pd.to_datetime(V['DateTime'])
        sig_t = (V['DateTime'] - s_time).dt.total_seconds()
        cols = V.columns
        df = V.loc[:,[ 'Vert Accelerometer','Lat Accelerometer','Long Accelerometer']]
        sig_v = np.sqrt(np.square(df.diff()).sum(axis=1))
        f = interpolate.interp1d(sig_t,sig_v,fill_value='extrapolate')
        new_sig = f(time_s)
        if pd.Series(new_sig).diff().abs().sum()/len(new_sig)>1: # since th
            df_sig.loc[:,PartName] = new_sig

In [ ]:
fig, (ax1) = plt.subplots(1, 1,figsize = [20,6])
sub_sig = df_sig
chart = sns.heatmap(data=df_sig.transpose(), ax=ax1,vmin=5, vmax=30, cbar=True,cmap="YlGnBu",alpha=0.8)

# chart = sns.heatmap(data=df_sig.transpose(), ax=ax1,vmin=40, vmax=120, cbar=True, cmap="YlGnBu",alpha=0.8)
#ax1.set_yticks([])
plt.title(projectName + ' ' + sigName + ' for interval ' + interval_name)
plt.savefig('./plots/'+projectName+'_'+ interval_name+ '_set_'+sigName+'.png',dpi = 300)
plt.show()

In [ ]:
#df_sig.index=pd.to_datetime(df_sig.index)
df_sig.index = time_dt
df_sig.index

In [ ]:
t1 = pd.to_datetime('2023-02-16 8:33:10.000+0000')
t2 = pd.to_datetime('2023-02-16 8:34:10.000+0000')
interval_name = 'Synch'

In [ ]:
sub_sig = df_sig.loc[t1:t2,:].copy()
sub_sig

In [ ]:
fig, (ax1) = plt.subplots(1, 1,figsize = [20,6])
chart = sns.heatmap(data=sub_sig.transpose(), ax=ax1,vmin=0, vmax=40, cbar=True,cmap="YlGnBu",alpha=0.8)

# chart = sns.heatmap(data=df_sig.transpose(), ax=ax1,vmin=40, vmax=120, cbar=True, cmap="YlGnBu",alpha=0.8)
#ax1.set_yticks([])
plt.title(projectName + ' ' + sigName + ' for interval ' + interval_name)
plt.savefig('./plots/'+projectName+'_'+ interval_name+ '_set_'+sigName+'.png',dpi = 300)
plt.show()

In [ ]:
df_sig.to_csv('./out_data/'+projectName+'_'+ interval_name+ '_set_'+sigName+'.csv')

In [ ]:
Beeps at 8:33:20
        so 3:20 = 200 s 
            
            